In [15]:
import pandas as pd
import glob

In [19]:
df = pd.concat([pd.read_csv(f, skiprows=2, on_bad_lines='skip') for f in glob.glob("data/Accounts_History*.csv")])

## Withdrawal Calculation from src/metrics.py

This shows the exact code used to calculate withdrawals in the dashboard:

In [ ]:
# First, we need to categorize transactions
def get_category(row):
    action = str(row['Action']).upper()
    description = str(row['Description']).upper()
    
    if "ELECTRONIC FUNDS TRANSFER" in action or "ELECTRONIC FUNDS TRANSFER" in description:
        return "DEPOSIT"
    elif "JOURNALED SPP PURCHASE CREDIT" in description or "JOURNALED SPP PURCHASE CREDIT" in action:
        return "DEPOSIT"
    elif "YOU BOUGHT" in action:
        return "BUY"
    elif "YOU SOLD" in action:
        return "SELL"
    elif "DIVIDEND" in action:
        return "DIVIDEND"
    elif "REINVESTMENT" in action:
        return "REINVESTMENT"
    elif "FOREIGN TAX" in action:
        return "TAX"
    elif "ADVISORY FEE" in action:
        return "FEE"
    else:
        return "OTHER"

df['Category'] = df.apply(get_category, axis=1)

# Calculate withdrawals (from calculate_net_invested_breakdown)
withdrawals = df[df['Category'] == 'WITHDRAWAL']['Amount'].sum()
print( df[df['Category'] == 'DEPOSIT']['Amount'].sum())
print(f"Total Withdrawals: ${withdrawals:,.2f}")
print(f"\nWithdrawal transactions:")
df[df['Category'] == 'WITHDRAWAL'][['Run Date', 'Action', 'Amount']]

49452.490000000005
Total Withdrawals: $0.00

Withdrawal transactions:


,Run Date,Action,Amount


## ESPP Credit and Usage Analysis

This shows ESPP credits received and how they were used to purchase MSFT stock:

In [23]:
# Find ESPP Credits
espp_credits = df[
    df['Description'].str.contains('JOURNALED SPP PURCHASE CREDIT', case=False, na=False) |
    df['Action'].str.contains('JOURNALED SPP PURCHASE CREDIT', case=False, na=False)
]

print("ESPP Credits Received:")
print(espp_credits[['Run Date', 'Action', 'Description', 'Amount']])
print(f"\nTotal ESPP Credits: ${espp_credits['Amount'].sum():,.2f}")

print("\n" + "="*80 + "\n")

# Find MSFT purchases using ESPP
espp_msft_purchases = df[
    (df['Category'] == 'BUY') & 
    (df['Symbol'] == 'MSFT') &
    (
        df['Description'].str.contains('ESPP', case=False, na=False) |
        df['Action'].str.contains('ESPP', case=False, na=False)
    )
]

print("MSFT Purchases Using ESPP:")
print(espp_msft_purchases[['Run Date', 'Action', 'Symbol', 'Quantity', 'Price', 'Amount']])
print(f"\nTotal ESPP used for MSFT purchases: ${espp_msft_purchases['Amount'].abs().sum():,.2f}")

print("\n" + "="*80 + "\n")
print("Summary:")
print(f"ESPP Credits: ${espp_credits['Amount'].sum():,.2f}")
print(f"ESPP Used for MSFT: ${espp_msft_purchases['Amount'].abs().sum():,.2f}")
print(f"Difference: ${espp_credits['Amount'].sum() - espp_msft_purchases['Amount'].abs().sum():,.2f}")

ESPP Credits Received:
      Run Date                                Action     Description   Amount
3   07/01/2025  JOURNALED SPP PURCHASE CREDIT (Cash)  No Description  5278.88
58  10/01/2025  JOURNALED SPP PURCHASE CREDIT (Cash)  No Description  6973.61

Total ESPP Credits: $12,252.49


MSFT Purchases Using ESPP:
      Run Date                                             Action Symbol  \
2   07/01/2025  YOU BOUGHT ESPP### AS OF 06-30-25 MICROSOFT CO...   MSFT   
57  10/01/2025  YOU BOUGHT ESPP### AS OF 09-30-25 MICROSOFT CO...   MSFT   

   Quantity    Price   Amount  
2       USD  11.7919 -5278.88  
57      USD  14.9597 -6973.61  

Total ESPP used for MSFT purchases: $12,252.49


Summary:
ESPP Credits: $12,252.49
ESPP Used for MSFT: $12,252.49
Difference: $0.00
